In [1]:
import math
from estimator import *
from sage.misc.table import table

In [2]:
jobs = 16

enable_binary = False

In [5]:
# LWE
n = 512
p = 2**6
q = 2048
e_stddev = 0.56  # For ternary

if 12 * e_stddev >= q / (2 * p):
    raise ValueError("noise standard deviation should be less than {}".format(q / (12 * 2 * p)))

e_lwe_distribution = ND.DiscreteGaussian(e_stddev)

if enable_binary:
    lwe_key_distribution = ND.SparseBinary(n // 2, n)
else:
    lwe_key_distribution = ND.SparseTernary(n // 4, n // 4, n)

In [6]:
lwe_param = LWE.Parameters(n=n, q=q, Xs=lwe_key_distribution, Xe=e_lwe_distribution)
print(lwe_param)
try:
    lwe_result = LWE.estimate(
        lwe_param,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    lwe_security = min([math.log(res.get("rop", 0), 2) for res in lwe_result.values()])
    header = ["n","q","secret key distribution","noise distribution", "lwe security"]
    data = [n,q,lwe_key_distribution,e_lwe_distribution,lwe_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=512, q=2048, Xs=D(σ=0.71), Xe=D(σ=0.56), m=+Infinity, tag=None)
Algorithm functools.partial(<function dual at 0x7ff615b14ae0>, red_cost_model=<estimator.reduction.MATZOV object at 0x7ff615e77910>) on LWEParameters(n=512, q=2048, Xs=D(σ=0.56), Xe=D(σ=0.56), m=+Infinity, tag=None) failed with Unknown how to exploit sparsity of D(σ=0.56, μ=0.00, n=512)
Algorithm arora_gb+guessing on LWEParameters(n=512, q=2048, Xs=D(σ=0.56), Xe=D(σ=0.56), m=+Infinity, tag=None) failed with 
bkw                  :: rop: ≈2^134.3, m: ≈2^121.0, mem: ≈2^114.5, b: 10, t1: 5, t2: 17, ℓ: 9, #cod: 392, #top: 0, #test: 71, tag: coded-bkw
usvp                 :: rop: ≈2^126.5, red: ≈2^126.5, δ: 1.004417, β: 344, d: 903, tag: usvp
bdd                  :: rop: ≈2^123.2, red: ≈2^122.4, svp: ≈2^121.8, β: 329, η: 360, d: 916, tag: bdd
dual_hybrid          :: rop: ≈2^120.1, red: ≈2^119.3, guess: ≈2^118.8, β: 314, p: 2, ζ: 20, t: 70, β': 322, N: ≈2^66.5, m: 512
┌─────────────────────────╥──────────────────

In [41]:
# RLWE
N = 1024
Q = 132120577
sigma_ring = 3.20 * (1 << 1)

if enable_binary:
    rlwe_key_distribution = ND.SparseBinary(N // 2, N)
else:
    rlwe_key_distribution = ND.SparseTernary(N // 4, N // 4, N)

e_rlwe_distribution = ND.DiscreteGaussian(sigma_ring)

logQ = Q.bit_length()

header = ["logQ"]
data = [logQ]
print(table(columns=[data], header_column=header, frame=True))

┌──────╥────┐
│ logQ ║ 27 |
└──────╨────┘


In [ ]:
rlwe_param = LWE.Parameters(n=N, q=Q, Xs=rlwe_key_distribution, Xe=e_rlwe_distribution)
print(rlwe_param)
try:
    # rlwe_result = LWE.estimate.rough(
    #     rlwe_param,
    #     jobs=jobs,
    # )
    # "arora-gb","bkw","usvp","bdd","bdd_hybrid","bdd_mitm_hybrid","dual","dual_hybrid","dual_mitm_hybrid"
    rlwe_result = LWE.estimate(
        rlwe_param,
        # red_shape_model=Simulator.CN11,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    rlwe_security = min([math.log(res.get("rop", 0), 2) for res in rlwe_result.values()])
    header = ["N","Q","logQ","secret key distribution","noise distribution", "rlwe security"]
    data = [N,Q,logQ,rlwe_key_distribution,e_rlwe_distribution,rlwe_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=1024, q=132120577, Xs=D(σ=0.71), Xe=D(σ=6.40), m=+Infinity, tag=None)
arora-gb             :: rop: ≈2^inf, m: ≈2^975.1, dreg: 411, t: 119, mem: ≈2^inf, tag: arora-gb, ↻: ≈2^727.7, ζ: 513, |S|: ≈2^637.8, prop: ≈2^-87.7
bkw                  :: rop: ≈2^233.6, m: ≈2^219.8, mem: ≈2^220.8, b: 8, t1: 0, t2: 31, ℓ: 7, #cod: 919, #top: 0, #test: 107, tag: coded-bkw
usvp                 :: rop: ≈2^133.0, red: ≈2^133.0, δ: 1.004266, β: 362, d: 1937, tag: usvp
bdd                  :: rop: ≈2^131.3, red: ≈2^131.1, svp: ≈2^128.2, β: 355, η: 383, d: 1940, tag: bdd
dual                 :: rop: ≈2^136.3, mem: ≈2^87.4, m: 984, β: 370, d: 2008, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^129.9, red: ≈2^129.9, guess: ≈2^120.0, β: 347, p: 3, ζ: 20, t: 40, β': 358, N: ≈2^73.2, m: 1024
┌─────────────────────────╥───────────────────────────┐
│ N                       ║ 1024                      |
├─────────────────────────╫───────────────────────────┤
│ Q                       ║ 132120577 

In [48]:
e_key_switching = 3.20 * (1 << 14)
key_switching_key_distribution = ND.DiscreteGaussian(e_key_switching)

In [49]:
key_switching = LWE.Parameters(n=n, q=Q, Xs=lwe_key_distribution, Xe=key_switching_key_distribution)
print(key_switching)
try:
    # key_switching_result = LWE.estimate.rough(
    #     key_switching,
    #     jobs=jobs,
    # )
    # "arora-gb","bkw","usvp","bdd","bdd_hybrid","bdd_mitm_hybrid","dual","dual_hybrid","dual_mitm_hybrid"
    key_switching_result = LWE.estimate(
        key_switching,
        # red_shape_model=Simulator.CN11,
        deny_list=(
            # "arora-gb",
            # "bkw",
        ),
        jobs=jobs,
    )
except Exception as err:
    print("err=", err)
    print("Error Occur!")
else:
    key_switching_security = min([math.log(res.get("rop", 0), 2) for res in key_switching_result.values()])
    header = ["n","logQ","secret key distribution","noise distribution", "key switching key security"]
    data = [n,logQ,lwe_key_distribution,key_switching_key_distribution,key_switching_security]
    print(table(columns = [data], header_column = header, frame = True))

LWEParameters(n=512, q=132120577, Xs=D(σ=0.71), Xe=D(σ=52428.80), m=+Infinity, tag=None)
bkw                  :: rop: ≈2^177.7, m: ≈2^165.3, mem: ≈2^166.3, b: 6, t1: 0, t2: 22, ℓ: 5, #cod: 467, #top: 0, #test: 45, tag: coded-bkw
usvp                 :: rop: ≈2^135.5, red: ≈2^135.5, δ: 1.004156, β: 376, d: 959, tag: usvp
bdd                  :: rop: ≈2^139.6, red: ≈2^124.5, svp: ≈2^139.6, β: 337, η: 424, d: 868, tag: bdd
dual                 :: rop: ≈2^141.1, mem: ≈2^90.8, m: 465, β: 393, d: 977, ↻: 1, tag: dual
dual_hybrid          :: rop: ≈2^129.6, red: ≈2^129.6, guess: ≈2^120.3, β: 351, p: 3, ζ: 20, t: 40, β': 357, N: ≈2^73.5, m: 512
┌────────────────────────────╥──────────────────────────┐
│ n                          ║ 512                      |
├────────────────────────────╫──────────────────────────┤
│ logQ                       ║ 27                       |
├────────────────────────────╫──────────────────────────┤
│ secret key distribution    ║ D(σ=0.71, μ=0.00, n=512) |
├───────